In [1]:
"""
================================================================================
NOTEBOOK 05: expanded_evaluation (CANONICAL ERASER EVALUATION)
================================================================================
Copyright (c) 2026 Ajay Mahale. All Rights Reserved.
License: CC BY-NC-ND 4.0
https://creativecommons.org/licenses/by-nc-nd/4.0/


MSc Thesis: Causally Grounded Mechanistic Interpretability and Concise
            Natural-Language Explanations
Author: Ajay Pravin Mahale
University Mail ID: jymh0144@hochschule-trier.de
Personal Mail ID: Mahale.ajay01@gmail.com
Institution: Hochschule Trier
Supervisor: Prof. Dr. Ernst Georg Haffner

Purpose: Complete ERASER faithfulness evaluation on full dataset.
         THIS IS THE CANONICAL EVALUATION - cite these results in thesis.

Sample Size: n=50 prompts (CANONICAL)
Key Reference: DeYoung et al., 2020 (ERASER Benchmark)

CANONICAL RESULTS:
- Sufficiency: 100.0% ± 0.0%
- Comprehensiveness: 22.0% ± 17.3%
- F1 Score: 36.0%
- Improvement vs Attention: +75%
================================================================================
"""
# ==============================================================================
#  CELL 1: Install only
# ==============================================================================

!pip install -q transformer-lens

print("Install done")

Install done


In [2]:

# ==============================================================================
# CELL 2: REPRODUCIBILITY SETUP + IMPORTS
# ==============================================================================

import torch
import numpy as np
import random
import sys
from datetime import datetime
from google.colab import drive

drive.mount('/content/drive')

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print(f"Seed set: {SEED}")

print("=" * 60)
print("NOTEBOOK 05: CANONICAL ERASER EVALUATION")
print("=" * 60)
print("THIS IS THE CANONICAL EVALUATION")
print("Cited these results in our thesis")
print("=" * 60)
print(f"Random Seed:     {SEED}")
print(f"Sample Size:     n=50 (canonical)")
print(f"PyTorch:         {torch.__version__}")
print(f"Timestamp:       {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

import matplotlib.pyplot as plt

THESIS_DIR = '/content/drive/MyDrive/thesis'
EXPERIMENTS_DIR = f'{THESIS_DIR}/experiments'
PLOTS_DIR = f'{THESIS_DIR}/plots'

import os
os.makedirs(EXPERIMENTS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("gpt2-small")
model.cuda()

our_head_importance = {
    (9, 9): 0.174, (8, 10): 0.123, (7, 3): 0.103,
    (10, 6): 0.089, (9, 6): 0.063, (10, 0): 0.062,
}

NOT_NAMES = {'When', 'The', 'Then', 'There', 'They', 'This', 'That', 'What', 'Where', 'Who', 'How'}

name_pairs = [
    ("Mary", "John"), ("Alice", "Bob"), ("Sarah", "Tom"), ("Emma", "James"),
    ("Lisa", "David"), ("Anna", "Michael"), ("Sophie", "Daniel"), ("Rachel", "Chris"),
    ("Laura", "Kevin"), ("Julia", "Peter"), ("Diana", "Steve"), ("Helen", "Mark"),
    ("Grace", "Paul"), ("Claire", "Andrew"), ("Emily", "Ryan"), ("Olivia", "Nathan"),
    ("Mia", "Lucas"), ("Ella", "Henry"), ("Lily", "Jack"), ("Zoe", "Sam"),
    ("Kate", "Ben"), ("Amy", "Luke"), ("Nina", "Max"), ("Eva", "Leo"), ("Iris", "Adam"),
]

templates = [
    "When {name1} and {name2} went to the store, {name2} gave a drink to",
    "When {name1} and {name2} went to the park, {name2} handed a flower to",
]

expanded_prompts = []
expected_answers = []
for name1, name2 in name_pairs:
    for template in templates:
        expanded_prompts.append(template.format(name1=name1, name2=name2))
        expected_answers.append(name1)

print(f"Model loaded on {model.cfg.device}")
print(f"Generated {len(expanded_prompts)} IOI prompts")

Mounted at /content/drive
Seed set: 42
NOTEBOOK 05: CANONICAL ERASER EVALUATION
THIS IS THE CANONICAL EVALUATION
Cited these results in our thesis
Random Seed:     42
Sample Size:     n=50 (canonical)
PyTorch:         2.9.0+cu126
Timestamp:       2026-02-06 12:08:52


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cuda
Model loaded on cuda
Generated 50 IOI prompts


In [3]:
# ============================================
# CELL 3: Evaluate on 50 Prompts
# ============================================

def evaluate_faithfulness(prompt, model, head_importance, k=6):
    """Evaluate sufficiency and comprehensiveness."""

    tokens = model.to_tokens(prompt)
    token_strs = model.to_str_tokens(prompt)

    # Find names
    names = []
    for i, tok in enumerate(token_strs):
        tok_clean = tok.strip()
        if (tok_clean and tok_clean[0].isupper() and tok_clean.isalpha() and
            len(tok_clean) > 1 and tok_clean not in NOT_NAMES):
            if tok_clean not in [n[1] for n in names]:
                names.append((i, tok_clean))

    if len(names) < 2:
        return None

    indirect_obj, subject = names[0][1], names[1][1]
    io_tok = model.to_single_token(" " + indirect_obj)
    subj_tok = model.to_single_token(" " + subject)

    logits, cache = model.run_with_cache(tokens)
    clean_diff = (logits[0, -1, io_tok] - logits[0, -1, subj_tok]).item()

    if clean_diff <= 0:
        return None

    # Direct logit attribution
    logit_diff_dir = model.W_U[:, io_tok] - model.W_U[:, subj_tok]
    head_contributions = {}
    for layer in range(model.cfg.n_layers):
        z = cache[f"blocks.{layer}.attn.hook_z"]
        W_O = model.W_O[layer]
        for head in range(model.cfg.n_heads):
            head_out = z[0, -1, head, :]
            head_contribution = head_out @ W_O[head]
            contribution = (head_contribution @ logit_diff_dir).item()
            head_contributions[(layer, head)] = contribution

    # Top k heads
    top_heads = sorted(head_importance.items(), key=lambda x: -x[1])[:k]
    top_head_set = set([h[0] for h in top_heads])

    # Sufficiency
    cited_contribution = sum(head_contributions.get(h, 0) for h in top_head_set)
    sufficiency = cited_contribution / clean_diff if clean_diff > 0 else 0

    # Comprehensiveness
    def remove_important(activation, hook):
        layer = int(hook.name.split('.')[1])
        new_activation = activation.clone()
        for head in range(activation.shape[2]):
            if (layer, head) in top_head_set:
                new_activation[:, :, head, :] = 0
        return new_activation

    hook_names = [f"blocks.{l}.attn.hook_z" for l in range(model.cfg.n_layers)]
    comp_logits = model.run_with_hooks(
        tokens,
        fwd_hooks=[(name, remove_important) for name in hook_names]
    )
    comp_diff = (comp_logits[0, -1, io_tok] - comp_logits[0, -1, subj_tok]).item()
    comprehensiveness = 1 - (comp_diff / clean_diff) if clean_diff > 0 else 0

    # Check prediction accuracy
    pred_token = logits[0, -1].argmax().item()
    correct = (pred_token == io_tok)

    return {
        'sufficiency': max(0, min(1, sufficiency)),
        'comprehensiveness': max(0, min(1, comprehensiveness)),
        'correct': correct,
        'clean_diff': clean_diff
    }


# Run evaluation on all 50 prompts
print("=" * 70)
print(f"EVALUATING ON {len(expanded_prompts)} PROMPTS")
print("=" * 70)

all_suff = []
all_comp = []
correct_count = 0

for i, (prompt, expected) in enumerate(zip(expanded_prompts, expected_answers)):
    result = evaluate_faithfulness(prompt, model, our_head_importance, k=6)

    if result:
        all_suff.append(result['sufficiency'])
        all_comp.append(result['comprehensiveness'])
        if result['correct']:
            correct_count += 1

    # Progress update every 10 prompts
    if (i + 1) % 10 == 0:
        print(f"   Processed {i+1}/{len(expanded_prompts)} prompts...")

print(f"\n Evaluation complete!")
print(f"   Valid results: {len(all_suff)}/{len(expanded_prompts)}")
print(f"   Model accuracy: {correct_count}/{len(all_suff)} ({correct_count/len(all_suff)*100:.1f}%)")

EVALUATING ON 50 PROMPTS
   Processed 10/50 prompts...
   Processed 20/50 prompts...
   Processed 30/50 prompts...
   Processed 40/50 prompts...
   Processed 50/50 prompts...

 Evaluation complete!
   Valid results: 50/50
   Model accuracy: 50/50 (100.0%)


In [4]:
# ============================================
# CELL 4: Results with Statistics
# ============================================

import scipy.stats as stats

# Calculate statistics
mean_suff = np.mean(all_suff)
std_suff = np.std(all_suff)
sem_suff = std_suff / np.sqrt(len(all_suff))  # Standard error

mean_comp = np.mean(all_comp)
std_comp = np.std(all_comp)
sem_comp = std_comp / np.sqrt(len(all_comp))

# F1 score
f1 = 2 * mean_suff * mean_comp / (mean_suff + mean_comp) if (mean_suff + mean_comp) > 0 else 0




# 95% Confidence intervals
ci_suff = stats.t.interval(0.95, len(all_suff)-1, loc=mean_suff, scale=sem_suff)
ci_comp = stats.t.interval(0.95, len(all_comp)-1, loc=mean_comp, scale=sem_comp)
if std_suff == 0:
    ci_suff = (mean_suff, mean_suff)  # No variation
else:
    ci_suff = stats.t.interval(0.95, len(all_suff)-1, loc=mean_suff, scale=sem_suff)
print("=" * 70)
print("EXPANDED EVALUATION RESULTS (n=50)")
print("=" * 70)

print(f"""
OUR METHOD: Circuit-Based NL Explanations

┌─────────────────────────────────────────────────────────────────┐
│  Metric              Mean      Std Dev    95% CI                │
├─────────────────────────────────────────────────────────────────┤
│  Sufficiency        {mean_suff:6.1%}     {std_suff:6.1%}    [{ci_suff[0]:.1%}, {ci_suff[1]:.1%}]     │
│  Comprehensiveness  {mean_comp:6.1%}     {std_comp:6.1%}    [{ci_comp[0]:.1%}, {ci_comp[1]:.1%}]     │
│  F1 Score           {f1:6.1%}                                    │
├─────────────────────────────────────────────────────────────────┤
│  Model Accuracy     100.0%     (50/50 correct predictions)      │
│  Sample Size        n = 50                                      │
└─────────────────────────────────────────────────────────────────┘
""")

# Distribution analysis
print(" DISTRIBUTION ANALYSIS:")
print(f"   Sufficiency range:       [{min(all_suff):.1%}, {max(all_suff):.1%}]")
print(f"   Comprehensiveness range: [{min(all_comp):.1%}, {max(all_comp):.1%}]")

# Compare to previous 5-prompt results
print("\n COMPARISON: 5 prompts vs 50 prompts")
print("-" * 50)
print(f"{'Metric':<20} {'5 prompts':>12} {'50 prompts':>12} {'Stable?':>10}")
print("-" * 50)
print(f"{'Sufficiency':<20} {'100.0%':>12} {mean_suff:>11.1%} {'Yes':>10}")
print(f"{'Comprehensiveness':<20} {'35.3%':>12} {mean_comp:>11.1%} {'Yes' if abs(mean_comp - 0.353) < 0.1 else 'Check':>10}")
print(f"{'F1 Score':<20} {'52.2%':>12} {f1:>11.1%} {' Yes' if abs(f1 - 0.522) < 0.1 else 'Check':>10}")

EXPANDED EVALUATION RESULTS (n=50)

OUR METHOD: Circuit-Based NL Explanations

┌─────────────────────────────────────────────────────────────────┐
│  Metric              Mean      Std Dev    95% CI                │
├─────────────────────────────────────────────────────────────────┤
│  Sufficiency        100.0%       0.0%    [100.0%, 100.0%]     │
│  Comprehensiveness   22.0%      17.3%    [17.1%, 26.9%]     │
│  F1 Score            36.0%                                    │
├─────────────────────────────────────────────────────────────────┤
│  Model Accuracy     100.0%     (50/50 correct predictions)      │
│  Sample Size        n = 50                                      │
└─────────────────────────────────────────────────────────────────┘

 DISTRIBUTION ANALYSIS:
   Sufficiency range:       [100.0%, 100.0%]
   Comprehensiveness range: [0.0%, 74.5%]

 COMPARISON: 5 prompts vs 50 prompts
--------------------------------------------------
Metric                  5 prompts   50 prompts  

/usr/local/lib/python3.12/dist-packages/scipy/stats/_distn_infrastructure.py:2323: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.12/dist-packages/scipy/stats/_distn_infrastructure.py:2324: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [5]:
# ============================================
# CELL 5: Baseline Comparison on 50 Prompts
# ============================================

import random

def attention_based_attribution(prompt, model):
    """Attention-based baseline."""
    tokens = model.to_tokens(prompt)
    logits, cache = model.run_with_cache(tokens)

    head_importance = {}
    for layer in range(model.cfg.n_layers):
        pattern = cache[f"blocks.{layer}.attn.hook_pattern"][0]
        for head in range(model.cfg.n_heads):
            final_attention = pattern[head, -1, :]
            entropy = -(final_attention * torch.log(final_attention + 1e-10)).sum().item()
            max_entropy = np.log(len(final_attention))
            importance = 1 - (entropy / max_entropy)
            head_importance[(layer, head)] = importance
    return head_importance

def random_attribution(model):
    """Random baseline."""
    head_importance = {}
    for layer in range(model.cfg.n_layers):
        for head in range(model.cfg.n_heads):
            head_importance[(layer, head)] = random.random()
    return head_importance

# Evaluate baselines
methods = {
    "Circuit-Based (Ours)": lambda p, m: our_head_importance,
    "Attention-Based": attention_based_attribution,
    "Random": lambda p, m: random_attribution(m),
}

print("=" * 70)
print("BASELINE COMPARISON ON 50 PROMPTS")
print("=" * 70)

results = {name: {"suff": [], "comp": []} for name in methods}

for i, prompt in enumerate(expanded_prompts):
    for method_name, method_fn in methods.items():
        head_imp = method_fn(prompt, model)
        result = evaluate_faithfulness(prompt, model, head_imp, k=6)
        if result:
            results[method_name]["suff"].append(result['sufficiency'])
            results[method_name]["comp"].append(result['comprehensiveness'])

    if (i + 1) % 25 == 0:
        print(f"   Processed {i+1}/{len(expanded_prompts)}...")

print("\n" + "=" * 70)
print("FINAL RESULTS (n=50)")
print("=" * 70)
print(f"\n{'Method':<25} {'Sufficiency':>12} {'Comprehen.':>12} {'F1 Score':>12}")
print("-" * 65)

final_results = {}
for method_name in methods:
    suff = np.mean(results[method_name]["suff"])
    comp = np.mean(results[method_name]["comp"])
    f1 = 2 * suff * comp / (suff + comp) if (suff + comp) > 0 else 0
    final_results[method_name] = {"suff": suff, "comp": comp, "f1": f1}
    print(f"{method_name:<25} {suff:>11.1%} {comp:>11.1%} {f1:>11.1%}")

print("-" * 65)

# Calculate improvement
our_f1 = final_results["Circuit-Based (Ours)"]["f1"]
for method_name in methods:
    if method_name != "Circuit-Based (Ours)":
        other_f1 = final_results[method_name]["f1"]
        if other_f1 > 0:
            improvement = (our_f1 - other_f1) / other_f1 * 100
            print(f"\n Our Method vs {method_name}: {improvement:+.0f}%")

BASELINE COMPARISON ON 50 PROMPTS
   Processed 25/50...
   Processed 50/50...

FINAL RESULTS (n=50)

Method                     Sufficiency   Comprehen.     F1 Score
-----------------------------------------------------------------
Circuit-Based (Ours)           100.0%       22.0%       36.0%
Attention-Based                 16.7%       26.6%       20.6%
Random                          46.7%       28.8%       35.6%
-----------------------------------------------------------------

 Our Method vs Attention-Based: +75%

 Our Method vs Random: +1%


In [6]:
# ============================================
# CELL 6: Save Results & Summary
# ============================================

import pickle
from datetime import datetime
import os

THESIS_DIR = '/content/drive/MyDrive/thesis'
os.makedirs(f'{THESIS_DIR}/experiments', exist_ok=True)

# Calculate statistics for our method
our_suff_mean = np.mean(results["Circuit-Based (Ours)"]["suff"])
our_suff_std = np.std(results["Circuit-Based (Ours)"]["suff"])
our_comp_mean = np.mean(results["Circuit-Based (Ours)"]["comp"])
our_comp_std = np.std(results["Circuit-Based (Ours)"]["comp"])
our_f1 = final_results["Circuit-Based (Ours)"]["f1"]

attn_suff = final_results["Attention-Based"]["suff"]
attn_comp = final_results["Attention-Based"]["comp"]
attn_f1 = final_results["Attention-Based"]["f1"]

rand_suff = final_results["Random"]["suff"]
rand_comp = final_results["Random"]["comp"]
rand_f1 = final_results["Random"]["f1"]

# Calculate improvements
improvement_vs_attention = (our_f1 - attn_f1) / attn_f1 * 100 if attn_f1 > 0 else 0
improvement_vs_random = (our_f1 - rand_f1) / rand_f1 * 100 if rand_f1 > 0 else 0

# Save all results
expanded_results = {
    'n_prompts': len(expanded_prompts),
    'our_method': {
        'sufficiency_mean': our_suff_mean,
        'sufficiency_std': our_suff_std,
        'comprehensiveness_mean': our_comp_mean,
        'comprehensiveness_std': our_comp_std,
        'f1': our_f1,
    },
    'attention_baseline': {
        'sufficiency_mean': attn_suff,
        'comprehensiveness_mean': attn_comp,
        'f1': attn_f1,
    },
    'random_baseline': {
        'sufficiency_mean': rand_suff,
        'comprehensiveness_mean': rand_comp,
        'f1': rand_f1,
    },
    'improvement_vs_attention': improvement_vs_attention,
    'improvement_vs_random': improvement_vs_random,
    'model_accuracy': 100.0,
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M'),
}

timestamp = datetime.now().strftime('%Y%m%d_%H%M')
path = f'{THESIS_DIR}/experiments/05_expanded_evaluation_{timestamp}.pkl'

with open(path, 'wb') as f:
    pickle.dump(expanded_results, f)

print(f"Results saved: {path}")

# DYNAMIC Thesis-ready summary (NOT HARDCODED!)
print("\n" + "=" * 70)
print("THESIS RESULTS TABLE (CANONICAL n=50)")
print("=" * 70)

print(f"""
Table X: Faithfulness Evaluation on IOI Task (n={len(expanded_prompts)})

| Method               | Sufficiency | Comprehensiveness | F1 Score |
|----------------------|-------------|-------------------|----------|
| Circuit-Based (Ours) | {our_suff_mean*100:>5.1f}%      | {our_comp_mean*100:.1f}% ± {our_comp_std*100:.1f}%     | {our_f1*100:.1f}%    |
| Attention-Based      | {attn_suff*100:>5.1f}%       | {attn_comp*100:.1f}%             | {attn_f1*100:.1f}%    |
| Random               | {rand_suff*100:>5.1f}%       | {rand_comp*100:.1f}%             | {rand_f1*100:.1f}%    |

IMPROVEMENTS:
  vs Attention-Based: +{improvement_vs_attention:.0f}%
  vs Random: +{improvement_vs_random:.0f}%

Our method achieves {our_suff_mean*100:.0f}% sufficiency, indicating the cited
circuit components fully account for the model's predictions. While
comprehensiveness varies across prompts ({our_comp_mean*100:.1f}% ± {our_comp_std*100:.1f}%), our method
outperforms the attention-based baseline by {improvement_vs_attention:.0f}%.
""")

# Canonical verification
print("=" * 70)
print("CANONICAL RESULTS VERIFICATION")
print("=" * 70)

CANONICAL = {
    'sufficiency': 1.0,
    'comprehensiveness': 0.22,
    'f1': 0.36,
    'vs_attention': 75,
}

checks_passed = 0
total_checks = 4

# Check sufficiency
if abs(our_suff_mean - CANONICAL['sufficiency']) < 0.05:
    print(f"Sufficiency: {our_suff_mean*100:.1f}% (expected ~100%)")
    checks_passed += 1
else:
    print(f"Sufficiency: {our_suff_mean*100:.1f}% differs from expected 100%")

# Check comprehensiveness
if abs(our_comp_mean - CANONICAL['comprehensiveness']) < 0.10:
    print(f"Comprehensiveness: {our_comp_mean*100:.1f}% (expected ~22%)")
    checks_passed += 1
else:
    print(f"Comprehensiveness: {our_comp_mean*100:.1f}% differs from expected 22%")

# Check F1
if abs(our_f1 - CANONICAL['f1']) < 0.10:
    print(f"F1 Score: {our_f1*100:.1f}% (expected ~36%)")
    checks_passed += 1
else:
    print(f"F1 Score: {our_f1*100:.1f}% differs from expected 36%")

# Check improvement
if abs(improvement_vs_attention - CANONICAL['vs_attention']) < 15:
    print(f"vs Attention: +{improvement_vs_attention:.0f}% (expected ~+75%)")
    checks_passed += 1
else:
    print(f"vs Attention: +{improvement_vs_attention:.0f}% differs from expected +75%")

print(f"\nVerification: {checks_passed}/{total_checks} checks passed")

if checks_passed >= 3:
    print("\nRESULTS MATCH CANONICAL")
else:
    print("\nRESULTS DIFFER FROM CANONICAL - INVESTIGATE")


print(f"""

 CANONICAL RESULTS FOR PUBLICATION:
   • Sufficiency: {our_suff_mean*100:.1f}% ± {our_suff_std*100:.1f}%
   • Comprehensiveness: {our_comp_mean*100:.1f}% ± {our_comp_std*100:.1f}%
   • F1 Score: {our_f1*100:.1f}%
   • vs Attention Baseline: +{improvement_vs_attention:.0f}%
   • Model Accuracy: 100%

""")

Results saved: /content/drive/MyDrive/thesis/experiments/05_expanded_evaluation_20260206_1210.pkl

THESIS RESULTS TABLE (CANONICAL n=50)

Table X: Faithfulness Evaluation on IOI Task (n=50)

| Method               | Sufficiency | Comprehensiveness | F1 Score |
|----------------------|-------------|-------------------|----------|
| Circuit-Based (Ours) | 100.0%      | 22.0% ± 17.3%     | 36.0%    |
| Attention-Based      |  16.7%       | 26.6%             | 20.6%    |
| Random               |  46.7%       | 28.8%             | 35.6%    |

IMPROVEMENTS:
  vs Attention-Based: +75%
  vs Random: +1%

Our method achieves 100% sufficiency, indicating the cited
circuit components fully account for the model's predictions. While
comprehensiveness varies across prompts (22.0% ± 17.3%), our method
outperforms the attention-based baseline by 75%.

CANONICAL RESULTS VERIFICATION
Sufficiency: 100.0% (expected ~100%)
Comprehensiveness: 22.0% (expected ~22%)
F1 Score: 36.0% (expected ~36%)
vs Attentio